In [1]:
import pandas as pd
import numpy as np

In [ ]:
!curl ipecho.net/plain

### Connect to Mysql

In [2]:
from pymysql import Connection
conn = Connection(
    host = '34.67.249.196',
    user = 'root',
    password = 'cs411047',
)
# print(conn.get_server_info)
# conn.close()

In [3]:
# use cursor and select database
cursor = conn.cursor()
conn.select_db("project")

### Create tables

In [16]:
cursor.execute('DROP TABLE IF EXISTS Airport')
cursor.execute('show tables')

3

In [17]:
# create table Airport
sql_tb_airport = '''CREATE TABLE Airport (
    IATA VARCHAR(10) PRIMARY KEY,   
    airport_name VARCHAR(50)
)'''
cursor.execute(sql_tb_airport)  

0

In [6]:
cursor.execute('DROP TABLE IF EXISTS Airline')
cursor.execute('show tables')

0

In [7]:
# create table Airline
sql_tb_airline = '''create table Airline(
    IATA            varchar(10) primary key,
    airline_name    varchar(100)
)'''
cursor.execute(sql_tb_airline)              

0

In [4]:
cursor.execute('DROP TABLE IF EXISTS Comment')
cursor.execute('show tables')

1

In [12]:
# create table Comment
sql_tb_Comment  = '''create table Comment(
    comment_id INTEGER PRIMARY KEY,
    text TEXT,
    user_name VARCHAR(100) NOT NULL,
    airline VARCHAR(100) NOT NULL,
    FOREIGN KEY (user_name) REFERENCES User(user_name) ON DELETE CASCADE,
    FOREIGN KEY (airline) REFERENCES Airline(IATA) ON DELETE CASCADE

)'''
cursor.execute(sql_tb_Comment)

0

In [8]:
cursor.execute('DROP TABLE IF EXISTS User')
cursor.execute('show tables')

1

In [9]:
# create table User
sql_tb_user  = '''CREATE TABLE User (
    user_name VARCHAR(100) PRIMARY KEY,
    password VARCHAR(100) NOT NULL,
    email VARCHAR(100),
    first_name VARCHAR(50),
    last_name VARCHAR(50)
)'''
cursor.execute(sql_tb_user)

0

In [10]:
cursor.execute('show tables')

2

In [63]:
cursor.execute('DROP TABLE IF EXISTS Ticket')
cursor.execute('show tables')

6

In [66]:
# create table Ticket
sql_tb_ticket  = '''
    CREATE TABLE Ticket (
    ticket_id VARCHAR(64) PRIMARY KEY,
    flight_id VARCHAR(64) NOT NULL,
    purchase_date DATE,
    CLASS VARCHAR(50),
    PRICE REAL,
    FOREIGN KEY (flight_id) REFERENCES Flight(flight_id) ON DELETE CASCADE
)'''
cursor.execute(sql_tb_ticket)

0

In [64]:
cursor.execute('DROP TABLE IF EXISTS Flight')
cursor.execute('show tables')

5

In [65]:
# create table Flight
sql_tb_flight  = '''
        CREATE TABLE Flight (
        flight_id VARCHAR(64) PRIMARY KEY,
        flight_number VARCHAR(30) NOT NULL,
        airline_code VARCHAR(10) NOT NULL,
        departure_date DATE,
        departure_time TIME,
        arrival_date DATE,
        arrival_time TIME,
        travel_time TIME,
        departure_airport VARCHAR(50),
        arrival_airport VARCHAR(50),
        FOREIGN KEY (airline_code) REFERENCES Airline(IATA) ON DELETE CASCADE,
        FOREIGN KEY (departure_airport) REFERENCES Airport(IATA) ON DELETE CASCADE,
        FOREIGN KEY (arrival_airport) REFERENCES Airport(IATA) ON DELETE CASCADE
)'''
cursor.execute(sql_tb_flight)

0

In [29]:
cursor.execute('DROP TABLE IF EXISTS Operate')
cursor.execute('show tables')

6

In [30]:
# create table Operate
sql_tb_operate  = '''
    CREATE TABLE Operate (
    airline_IATA VARCHAR(10),
    airport_IATA VARCHAR(10),
    PRIMARY KEY (airline_IATA, airport_IATA),
    FOREIGN KEY (airline_IATA) REFERENCES Airline(IATA),
    FOREIGN KEY (airport_IATA) REFERENCES Airport(IATA)

)'''
cursor.execute(sql_tb_operate)

0

In [25]:
conn.close()

### Insert values into table Airline

In [31]:
df = pd.read_csv("E:\\UIUC\\CS411 Database Systems\\fa22-cs411-A-team047-Pepsi\data\\airline_IATA.txt", sep="\t", header=None, 
                 names=["AIRLINE NAME", "IATA DESIGNATOR", "3 DIGIT CODE", "ICAO CODE", "COUNTRY / TERRITORY"])
df.head()

,AIRLINE NAME,IATA DESIGNATOR,3 DIGIT CODE,ICAO CODE,COUNTRY / TERRITORY
0,ABX Air,GB,832,ABX,United States
1,Aegean Airlines,A3,390,AEE,Greece
2,Aer Lingus,EI,053,EIN,Ireland
3,Aero Republica,P5,845,RPB,Colombia
4,Aeroflot,SU,555,AFL,Russian Federation


In [32]:
# remove duplicated vale "LH"
LH_index = df['IATA DESIGNATOR']=="LH"
df.drop(index = [190],inplace = True)

In [34]:
# to csv
df.to_csv('./airline.csv')

In [51]:
df[df["IATA DESIGNATOR"]=="ZQ"]

,AIRLINE NAME,IATA DESIGNATOR,3 DIGIT CODE,ICAO CODE,COUNTRY / TERRITORY
141,German Airways,ZQ,944,GER,Germany


In [58]:
df.shape

(296, 5)

In [52]:
df[135:150]

,AIRLINE NAME,IATA DESIGNATOR,3 DIGIT CODE,ICAO CODE,COUNTRY / TERRITORY
135,FLYONE,5F,130,FIA,"Moldova, Republic of"
136,Freebird Airlines,FH,None,FHY,Turkey
137,French Bee,BF,396,FBU,France
138,Fuzhou Airlines,FU,666,FZA,China (People's Republic of)
139,Garuda Indonesia,GA,126,GIA,Indonesia
140,Georgian Airways,A9,606,TGZ,Georgia
141,German Airways,ZQ,944,GER,Germany
142,GOL Linhas Aereas,G3,127,GLO,Brazil
143,Gulf Air,GF,072,GFA,Bahrain
144,GX Airlines,GX,872,CBG,China (People's Republic of)


In [ ]:
# Insert DataFrame recrds one by one.
for i,row in df.iterrows():
    IATA = row['IATA DESIGNATOR']
    airline_name = row['AIRLINE NAME']
    sql = '''INSERT INTO Airline VALUES ('%s','%s')'''%(IATA, airline_name)
    cursor.execute(sql)

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()
conn.close()

In [58]:
# change mm-dd-yyyy to yyyy-mm-dd in flight csv
flight = pd.read_csv("E:\\UIUC\\CS411 Database Systems\\fa22-cs411-A-team047-Pepsi\\data\\flight.csv")
flight.head()

,flight_id,flight_number,airline_code,departure_date,departure_time,arrival_date,arrival_time,travel_time,departure_airport,arrival_airport
0,c45645fc442926434e54d4ad71966a4f7cd2420ccf7a8f...,NK 1024,NK,2022-10-21,21:59,2022-10-22,00:39,4:40:00,ORD,LAX
1,909459dcc544dec6216b232c0426e4e5dba71aae70214e...,UA 758,UA,2022-10-21,10:35,2022-10-21,13:00,4:25:00,ORD,LAX
2,9cd975325830680a6189efaa693a96c3fed69433d69ddb...,AA 2364,AA,2022-10-21,17:00,2022-10-21,19:16,4:16:00,ORD,LAX
3,5c690cbaed69c4ac2b03b1fe10aa91a9b0eb0b4ba84c3f...,WN 1766,WN,2022-10-21,12:25,2022-10-21,14:50,4:25:00,MDW,LAX
4,dfb9db781edd1b288b42cf5c7a15c833d94381f4328148...,UA 2676,UA,2022-10-21,14:12,2022-10-21,16:43,4:31:00,ORD,LAX


In [59]:
flight = flight.drop_duplicates(subset='flight_id')

In [61]:
flight.to_csv('./flight.csv')